In [ ]:
## Imports and seed
import os
import glob
import random
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader

# Set random seed for reproducibility
torch.manual_seed(22)
np.random.seed(22)
random.seed(22)

In [ ]:
## Custom Dataset for Right-Leg Gait Phase Estimation (CSV version)
class GaitPhaseDataset(Dataset):
    def __init__(self, root_dir, sequence_length=128, subjects=None, transform=None):
        """
        Args:
            root_dir (str): Root directory of the dataset. Files are expected under:
                dataset/<subject>/treadmill/imu/*.csv
                and corresponding gait cycle files under:
                dataset/<subject>/treadmill/gcRight/*.csv.
            sequence_length (int): Number of time steps in each sample window.
            subjects (list of str or None): List of subject IDs (e.g., ['AB09', 'AB10']).
                If None, all subjects are used.
            transform (callable, optional): Optional transform to be applied on the IMU window.
        """
        self.root_dir = root_dir
        self.sequence_length = sequence_length
        self.transform = transform

        # Locate all treadmill IMU CSV files.
        # search_path = os.path.join(root_dir, '*', 'treadmill', 'imu', '*.csv')
        # self.imu_files = glob.glob(search_path, recursive=True)
        if subjects is not None:
            # Filter files based on subject IDs
            filtered_files = []
            for subject in subjects:
                subject_path = os.path.join(root_dir, subject, '*', 'treadmill', 'imu', '*.csv')
                filtered_files.extend(glob.glob(subject_path))
            self.imu_files = filtered_files
            print("Sample paths:")
        if len(self.imu_files) == 0:
            raise RuntimeError("No IMU files found. Please check your dataset directory and folder structure.")

    
    def __len__(self):
        return len(self.imu_files)
    
    def __getitem__(self, idx):
        # Get the IMU CSV file path.
        imu_path = self.imu_files[idx]
        # Derive the corresponding gcRight CSV file path by replacing 'imu' with 'gcRight'
        gcRight_path = imu_path.replace(os.sep + 'imu' + os.sep, os.sep + 'gcRight' + os.sep)
        
        # Load CSV files (skip the header row)
        imu_data = self._load_csv_file(imu_path)
        gcRight_data = self._load_csv_file(gcRight_path)
        
        # Drop the timestamp column (first column)
        imu_data = imu_data[:, 1:]
        gcRight_data = gcRight_data[:, 1:]
        
        # Select only shank and thigh channels from IMU data.
        # CSV column order (after dropping timestamp) is:
        # We keep shank (columns 6 to 11) and thigh (columns 12 to 17)
        shank = imu_data[:, 6:12]
        thigh = imu_data[:, 12:18]
        imu_selected = np.concatenate([shank, thigh], axis=1)  # Shape: (N, 12)
        
        # Synchronize lengths: truncate all signals to the minimum available length.
        min_length = min(imu_selected.shape[0], gcRight_data.shape[0])
        imu_selected = imu_selected[:min_length, :]
        gcRight_data = gcRight_data[:min_length, :]
        
        end_idx = start_idx + self.sequence_length
        imu_window = imu_selected[start_idx:end_idx, :]  # (sequence_length, 12)
        
        # Use the HeelStrike value from gcRight at the center of the window.
        center_idx = start_idx + self.sequence_length // 2
        heel_strike = gcRight_data[center_idx, 0]  # HeelStrike value (0-100)
        # Normalize to [0, 1]
        heel_strike_norm = heel_strike / 100.0
        target = np.array([heel_strike_norm], dtype=np.float32)
        
        # Optionally apply a transform; otherwise, convert to torch tensors.
        if self.transform:
            imu_window = self.transform(imu_window)
        else:
            imu_window = torch.tensor(imu_window, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)
        
        return imu_window, target

    def _load_csv_file(self, file_path):
        """Loads a CSV file using NumPy (skipping the header row)."""
        data = np.loadtxt(file_path, delimiter=',', skiprows=1)
        return data

In [ ]:
## Baseline CNN Model for Gait Phase Estimation (Regression)
class BaselineGaitPhaseCNN(nn.Module):
    def __init__(
        self,
        num_channels=12,      # Number of input channels (e.g., 12 for shank+thigh IMU)
        sequence_length=128,  # Input time window length
        output_dim=1,         # Regression output dimension (e.g., 1 for gait phase)
        conv_filters=[32, 64],# Filters for each conv block (2 blocks in this example)
        kernel_size=3,        # Kernel size for all conv layers
        stride=1,             # Stride for all conv layers
        padding=0,            # Padding for all conv layers
        dilation=1,           # Dilation for all conv layers
        pool_size=2,          # Max-pooling factor
        hidden_units=100,     # Units in the first fully connected layer
        dropout_rate=0.5,     # Dropout probability
        activation='relu'     # Activation function: 'relu', 'sigmoid', or 'tanh'
    ):
        super(BaselineGaitPhaseCNN, self).__init__()
        
        self.activation_choice = activation.lower()
        
        self.conv_blocks = nn.ModuleList()
        
        # Track current number of channels and current sequence length.
        in_channels = num_channels
        L = sequence_length

        # Function to calculate output length after a 1D convolution (taking dilation into account)
        # Formula: L_out = floor((L_in + 2*padding - dilation*(kernel_size - 1) - 1)/stride + 1)
        def conv_output_length(L_in, kernel_size, stride, padding, dilation):
            return (L_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

        # Helper function to choose an activation function
        def get_activation_fn(act):
            if act == 'relu':
                return nn.ReLU()
            elif act == 'sigmoid':
                return nn.Sigmoid()
            elif act == 'tanh':
                return nn.Tanh()
            else:
                raise ValueError("Unsupported activation function. Choose 'relu', 'sigmoid', or 'tanh'.")
        
        act_fn = get_activation_fn(self.activation_choice)
        
        # Build each convolutional block
        for out_channels in conv_filters:
            # Convolution + BatchNorm + Activation + MaxPool
            block = nn.Sequential(
                nn.Conv1d(in_channels, out_channels,
                          kernel_size=kernel_size,
                          stride=stride,
                          padding=padding,
                          dilation=dilation),
                nn.BatchNorm1d(out_channels),
                act_fn,
                nn.MaxPool1d(pool_size)
            )
            self.conv_blocks.append(block)
            
            # Update sequence length after convolution and pooling
            L_conv = conv_output_length(L, kernel_size, stride, padding, dilation)
            L_pool = L_conv // pool_size
            L = L_pool
            
            # Update for next block
            in_channels = out_channels
        
        # Flattened size after all convolutional blocks
        flattened_size = in_channels * L
        
        # Fully connected layers
        self.fc1 = nn.Linear(flattened_size, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_units, output_dim)
        
        # Activation for FC layers using the same activation choice
        self.fc_activation = get_activation_fn(self.activation_choice)
        
    def forward(self, x):
        # x shape: (batch_size, sequence_length, num_channels)
        # Rearranged to (batch_size, num_channels, sequence_length) for Conv1d
        x = x.transpose(1, 2)
        
        # Pass through each convolutional block
        for block in self.conv_blocks:
            x = block(x)
        
        # Flatten the features
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = self.fc_activation(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Note on batch size:
# The batch size is set when creating a DataLoader. For example:
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# You can easily tune the batch size by changing that value.



In [ ]:
## Training and Evaluation Functions (Regression)
def train_model(model, device, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience=5):
    """
    Train the model with early stopping and return the best model along with loss histories.
    """
    best_model_weights = copy.deepcopy(model.state_dict())
    best_val_loss = float('inf')
    train_loss_history = []
    val_loss_history = []
    epochs_since_improvement = 0
    
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * data.size(0)
        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_loss_history.append(epoch_train_loss)
        
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for data, targets in val_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * data.size(0)
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_loss_history.append(epoch_val_loss)
        
        scheduler.step()
        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")
        
        # Early stopping: if no improvement for 'patience' epochs, break.
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_weights = copy.deepcopy(model.state_dict())
            epochs_since_improvement = 0
        else:
            epochs_since_improvement += 1
            
        if epochs_since_improvement >= patience:
            print(f"Early stopping triggered after {patience} epochs without improvement.")
            break

    model.load_state_dict(best_model_weights)
    return model, train_loss_history, val_loss_history

def test_model(model, device, test_loader, criterion):
    """
    Evaluate the model on the test set and return the average test loss.
    """
    model.eval()
    running_test_loss = 0.0
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            loss = criterion(outputs, targets)
            running_test_loss += loss.item() * data.size(0)
    test_loss = running_test_loss / len(test_loader.dataset)
    return test_loss


In [ ]:
#%% Main: One-Subject-Out Cross-Validation Setup and Training
if __name__ == "__main__":
    # Define dataset root directory
    dataset_root = r"data"  # Update with your actual data directory

    # Get list of subject folders
    all_subjects = [d for d in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, d))]
    all_subjects.sort()
    print("Subjects found:", all_subjects)

    # One-subject-out split (for demonstration; loop over all subjects for full CV)
    test_subject = all_subjects[0]
    train_subjects = all_subjects[1:]
    print(f"\nTraining on subjects: {train_subjects}")
    print(f"Testing on subject: {test_subject}")

    # Hyperparameters
    sequence_length = 128
    batch_size = 128
    num_epochs = 30
    learning_rate = 1e-3
    dropout_rate = 0.5
    patience = 5  # Early stopping patience

    # Create training and validation datasets (80/20 split)
    train_dataset_full = GaitPhaseDataset(root_dir=dataset_root, sequence_length=sequence_length, subjects=train_subjects)
    num_train = int(0.8 * len(train_dataset_full))
    num_val = len(train_dataset_full) - num_train
    train_dataset, val_dataset = random_split(train_dataset_full, [num_train, num_val])

    # Create test dataset (from test_subject)
    test_dataset = GaitPhaseDataset(root_dir=dataset_root, sequence_length=sequence_length, subjects=[test_subject])

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Hyperparameter tuning for optimizer and loss function:
    hyperparams = {
        "optimizer": "adam",       # Options: "adam", "sgd"
        "loss_function": "mse",      # Options: "mse", "mae", "huber"
    }

    # Instantiate the model
    model = BaselineGaitPhaseCNN(num_channels=12, sequence_length=sequence_length, output_dim=1, dropout_rate=dropout_rate)
    
    # Choose the loss function
    loss_fn = hyperparams["loss_function"].lower()
    if loss_fn == "mse":
        criterion = nn.MSELoss()
    elif loss_fn == "mae":
        criterion = nn.L1Loss()
    elif loss_fn == "huber":
        criterion = nn.SmoothL1Loss()
    else:
        raise ValueError("Unsupported loss function. Choose 'mse', 'mae', or 'huber'.")

    # Choose the optimizer
    optim_choice = hyperparams["optimizer"].lower()
    if optim_choice == "adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optim_choice == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer. Choose 'adam' or 'sgd'.")

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Running on device:", device)

    # Train the model with early stopping
    model, train_loss_hist, val_loss_hist = train_model(
        model, device, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience=patience
    )

    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_loss_hist, label='Train Loss')
    plt.plot(val_loss_hist, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Evaluate on test set
    test_loss = test_model(model, device, test_loader, criterion)
    print(f"Test Loss (MSE): {test_loss:.4f}")
